# Install Package

# Import Module

In [2]:
# 기본 패키지
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

# 정규화 패키지
import re

# 모델링 패키지
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# NLP 패키지
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경고무시
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [4]:
df = pd.read_csv('Petition_data_english.csv')
df.head()

,content,label,content_eng
0,AI 이미지 생성기의 무분별한 사용과 악용을 막기 위한 법적 규제에 관한 청원 AI...,1,The development of petition AI image generator...
1,아동학대살인 가해자의 엄벌과 신상공개에 관한 청원 저는 얼마전 아동학대로 살해당한 ...,1,A petition for severe punishment and disclosur...
2,중도금 가산금리 인하 및 시스템 개편에 관한 청원 정부에서 지역별 중도금 가산금리에...,1,Do you know that the petition government on th...
3,"한전은 공기업, 송전시장 민영화 반대에 관한 청원 자금난으로 인한 민간에 송전시장을...",1,KEPCO's petition against privatization of publ...
4,"12년간 당한 학교폭력에 관한 청원 저는 OO초등학교 2003년 입학, OO초등학교...",1,I entered OO Elementary School in 2003 and tra...


# Data Preprocessing

In [5]:
import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    
    # 특수문자 제거
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    # 숫자 제거
    text = re.sub('\w*\d\w*', '', text)
    
    # 불용어 처리
    words = text.split()
    words = [word for word in words if not word in stop_words]
    text = " ".join(words)
    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
df['tokenized'] = df['content'].apply(preprocess_text)
df.head()

,content,label,content_eng,tokenized
0,AI 이미지 생성기의 무분별한 사용과 악용을 막기 위한 법적 규제에 관한 청원 AI...,1,The development of petition AI image generator...,ai 이미지 생성기의 무분별한 사용과 악용을 막기 위한 법적 규제에 관한 청원 ai...
1,아동학대살인 가해자의 엄벌과 신상공개에 관한 청원 저는 얼마전 아동학대로 살해당한 ...,1,A petition for severe punishment and disclosur...,아동학대살인 가해자의 엄벌과 신상공개에 관한 청원 저는 얼마전 아동학대로 살해당한 ...
2,중도금 가산금리 인하 및 시스템 개편에 관한 청원 정부에서 지역별 중도금 가산금리에...,1,Do you know that the petition government on th...,중도금 가산금리 인하 및 시스템 개편에 관한 청원 정부에서 지역별 중도금 가산금리에...
3,"한전은 공기업, 송전시장 민영화 반대에 관한 청원 자금난으로 인한 민간에 송전시장을...",1,KEPCO's petition against privatization of publ...,한전은 공기업 송전시장 민영화 반대에 관한 청원 자금난으로 인한 민간에 송전시장을 ...
4,"12년간 당한 학교폭력에 관한 청원 저는 OO초등학교 2003년 입학, OO초등학교...",1,I entered OO Elementary School in 2003 and tra...,당한 학교폭력에 관한 청원 저는 oo초등학교 입학 oo초등학교에서 무렵 oooo초등...


#### Train, Test 분리

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df['tokenized'], df['label'], stratify = df['label'], random_state = 42, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((431,), (108,), (431,), (108,))

#### 토크나이징 및 패딩작업 진행

In [26]:
def preprocess(df):
    
    # 전처리된 문장에서 토큰 생성
    tokenizer = Tokenizer(oov_token='<UNK>')
    tokenizer.fit_on_texts(df)
    tokens = tokenizer.texts_to_sequences(df)

    df_wordcount = pd.DataFrame(list(tokenizer.word_counts.items()), columns=['word', 'count'])
    wc = df_wordcount[df_wordcount['count']<2].shape[0]
    vocab_size = len(tokenizer.word_index) - wc + 2
    print('단어 집합의 크기:', vocab_size)
    
    # 패딩
    max_seq_len = max([len(token_list) for token_list in tokens])
    padded_tokens = pad_sequences(tokens, maxlen=400, padding='post')
    print(f'max len: {max_seq_len}')

    return padded_tokens

In [27]:
X_train = preprocess(X_train)
X_test = preprocess(X_test)

단어 집합의 크기: 11920
max len: 429
단어 집합의 크기: 3402
max len: 406


# Modeling

In [29]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

In [30]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm

In [31]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.pos_emb = tf.keras.layers.Embedding(max_len, embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [32]:
max_len, vocab_size = 400, 11920
embedding_dim = 32  # 각 단어의 임베딩 벡터의 차원
num_heads = 2  # 어텐션 헤드의 수
dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 400)]             0         
                                                                 
 token_and_position_embeddin  (None, 400, 32)          394240    
 g_2 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_block_2 (Transf  (None, 400, 32)          6464      
 ormerBlock)                                                     
                                                                 
 global_average_pooling1d_2   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_10 (Dropout)        (None, 32)                0   

In [33]:
earlystopping = EarlyStopping(monitor="val_loss", patience = 5)

checkpoint = ModelCheckpoint('transformer_eng.h5',
                             save_best_only=True,
                             save_weights_only=True,
                             monitor='val_loss',
                             mode='min',
                             verbose=False)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, callbacks=[checkpoint, earlystopping], batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
14/14 [==============================] - 8s 246ms/step - loss: 0.5414 - accuracy: 0.7958 - val_loss: 0.4836 - val_accuracy: 0.8241
Epoch 2/50
14/14 [==============================] - 3s 221ms/step - loss: 0.4710 - accuracy: 0.8144 - val_loss: 0.4715 - val_accuracy: 0.8241
Epoch 3/50
14/14 [==============================] - 3s 200ms/step - loss: 0.4468 - accuracy: 0.8190 - val_loss: 0.4749 - val_accuracy: 0.8241
Epoch 4/50
14/14 [==============================] - 3s 215ms/step - loss: 0.3982 - accuracy: 0.8237 - val_loss: 0.4773 - val_accuracy: 0.8241
Epoch 5/50
14/14 [==============================] - 3s 219ms/step - loss: 0.3650 - accuracy: 0.8422 - val_loss: 0.5120 - val_accuracy: 0.8241
Epoch 6/50
14/14 [==============================] - 3s 218ms/step - loss: 0.2904 - accuracy: 0.8956 - val_loss: 0.5965 - val_accuracy: 0.8241
Epoch 7/50
14/14 [==============================] - 3s 185ms/step - loss: 0.1670 - accuracy: 0.9513 - val_loss: 0.6406 - val_accuracy: 0.8148


In [34]:
# 저장된 checkpoint 로드 후 정확도 측정
model.load_weights('transformer_eng.h5')
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 7ms/step - loss: 0.4715 - accuracy: 0.8241


[0.47153380513191223, 0.8240740895271301]

In [35]:
model.predict(X_test)[0]

4/4 [==============================] - 0s 6ms/step


array([0.16640498], dtype=float32)